Format the log sequences into instruction-response pairs.

In [1]:
import sys
print(sys.executable)
import torch
import transformers
print(torch.__version__)
print(transformers.__version__)

/Users/22.eric.oh/opt/anaconda3/envs/llm-env/bin/python


/Users/22.eric.oh/opt/anaconda3/envs/llm-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.2.0
4.51.0


In [2]:
import json
import random
from datetime import datetime, timedelta
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
from trl import SFTTrainer

def format_training(lines):
    return {
        "prompt": "Generate a realistic sequence of application logs showing normal operations with occasional errors:",
        "completion": "\n".join(lines)
    }

examples = []
with open("markov_flow.log", 'r') as f:
    lines = f.readlines()

chunk_size = 100
for i in range(0, len(lines) - chunk_size, chunk_size // 2):
    chunk = lines[i : i + chunk_size]
    examples.append(format_training([line.strip() for line in chunk]))

with open("training_data.jsonl", 'w') as f:
    for example in examples:
        f.write(json.dumps(example) + "\n")


RuntimeError: Failed to import trl.trainer.sft_trainer because of the following error (look up to see its traceback):
No module named 'rich'

In [5]:
# 1) Load bae model and tokenizer

model_name = "Qwen/Qwen3-4B-Instruct-2507"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype = torch.float16,
    device_map = "auto"
)

# 2) Configure LoRA. The layers we are actually training.

lora_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    r = 16,
    lora_alpha = 32,
    lora_dropout = 0.05,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# 3) Load the dataset
dataset = load_dataset("json", data_files="training_data.jsonl", split="train")

# 4) Format each example into a single string the model trains on
def format_example(example):
    return {
        "text" : f"### Instruction:\n{example['prompt']}\n\n### Response:\n{example['completion']}"
    }

dataset = dataset.map(format_example)

# 5) Training configuration
training_args = TrainingArguments(
    output_dir="./log-gen-model",
    num_train_epochs = 3,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    learning_rate = 2e-4,
    fp16 = True,
    logging_steps = 10,
    save_steps = 100,
    save_total_limit = 2,
)

# 6) Train
trainer = SFTTrainer(
    model = model,
    args = training_args,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 512,
)

trainer.train()

# 7) Save the final LoRA weights
model.save_pretrained("./log-gen-lora-weights")
tokenizer.save_pretrained("./log-gen-lora-weights")
print("Training complete. Weights saved to ./log-gen-lora-weights")

Exception: data did not match any variant of untagged enum ModelWrapper at line 757479 column 3

In [ ]:
# Load base model
base_model_name = "Qwen/Qwen3-4B-Instruct-2507"
tokenizer = AutoTokenizer.from_pretrained("./log-gen-lora-weights")
base_model = AutoModelFromCausalLM.from_pretrained(
    base_model_name,
    torch_dtype = torch.float16,
    device_map = "auto"
)

model = PeftModel.from_pretrained(base_model, "./log-gen-lora-weights")
model.eval()

# Generate logs
def generate_logs(num_lines = 50):
    prompt = """### Instruction: Generate a realistic sequence of application logs showing normal operations with occasional errors: ### Response:"""

    inputs = tokenizer(prompt, return_tensors = "pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_token = num_lines * 80,
            temperature = 0.8,
            top_p = 0.95,
            do_sample = True,
            repetition_penalty = 1.1
        )

    response = tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens = True
    )

    return response

print(generate_logs(50))